In [1]:
import scipy as sp
import numpy as np

---

In [2]:
import numpy as np
from typing import List, Set, Tuple

def construct_constraint_matrices(
    constraints: List[Set[Tuple[int, int]]],
    values: List[float],
    m: int,  # Number of input rows
    n: int   # Number of input columns
) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    """
    Construct Q, G, and W matrices for multiregion constraints
    
    Parameters:
    -----------
    constraints : List[Set[Tuple[int, int]]]
        List of index pair sets for each constraint
    values : List[float]
        Corresponding values for each constraint set
    m : int
        Number of rows in the input matrix X
    n : int
        Number of columns in the input matrix X
    
    Returns:
    --------
    Tuple[np.ndarray, np.ndarray, np.ndarray]
        Q (rows aggregation matrix, M x m), 
        G (columns aggregation matrix, n x N), 
        W (constraint values matrix, M x N)
    """
    # Find unique constraint indices
    first_components = [(set(idx[0] for idx in constraint_set)) for constraint_set in constraints]
    second_components = [(set(idx[1] for idx in constraint_set)) for constraint_set in constraints]
    
    # Determine M and N by finding groups of first and second components
    def find_groups(components):
        groups = []
        for comp_set in components:
            matching_groups = [group for group in groups if not group.isdisjoint(comp_set)]
            if matching_groups:
                merged_group = set.union(*matching_groups, comp_set)
                groups = [group for group in groups if group not in matching_groups]
                groups.append(merged_group)
            else:
                groups.append(comp_set)
        return groups

    row_groups = find_groups(first_components)
    col_groups = find_groups(second_components)

    # Ensure G @ 1 = 1 and 1.T @ Q = 1.T
    covered_rows = {i for group in row_groups for i in group}
    covered_cols = {j for group in col_groups for j in group}
    
    if len(covered_rows) < m:
        uncovered_rows = set(range(m)) - covered_rows
        row_groups.append(uncovered_rows)

    if len(covered_cols) < n:
        uncovered_cols = set(range(n)) - covered_cols
        col_groups.append(uncovered_cols)

    # Dimensions
    M = len(row_groups)
    N = len(col_groups)

    # Initialize Q, G, and W matrices
    G = np.zeros((M, m), dtype=float)
    Q = np.zeros((n, N), dtype=float)
    W = np.full((M, N), np.nan, dtype=float)  # W defaults to NaN

    # Create mapping for row and column groups
    row_group_map = {}
    for group_idx, group in enumerate(row_groups):
        for row in group:
            row_group_map[row] = group_idx

    col_group_map = {}
    for group_idx, group in enumerate(col_groups):
        for col in group:
            col_group_map[col] = group_idx

    # Fill G matrix (row aggregation)
    for row, group_idx in row_group_map.items():
        G[group_idx, row] = 1.0

    # Fill Q matrix (column aggregation)
    for col, group_idx in col_group_map.items():
        Q[col, group_idx] = 1.0

    # Fill W matrix with constraint values
    for constraint_set, value in zip(constraints, values):
        for i, j in constraint_set:
            row_idx = row_group_map[i]
            col_idx = col_group_map[j]
            W[row_idx, col_idx] = value

    return G, Q, W


In [5]:
constraints = [
    {(0, 4), (1, 5), (2, 3)},  # First constraint set
    {(3, 4), (4, 5), (5, 3)},  # Second constraint set
]
    
# Corresponding values for constraints
values = [10.0, 20.0]

# Specify input matrix dimensions
m, n = 7, 6  # Dimensions of input matrix X

# Construct matrices
G, Q, W = construct_constraint_matrices(constraints, values, m, n)

print("\nG matrix (row aggregation):")
print(G)
print("Q matrix (column aggregation):")
print(Q)
print("\nW matrix (constraint values):")
print(W)
    


G matrix (row aggregation):
[[1. 1. 1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 1. 1. 0.]
 [0. 0. 0. 0. 0. 0. 1.]]
Q matrix (column aggregation):
[[0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]]

W matrix (constraint values):
[[10. nan]
 [20. nan]
 [nan nan]]


In [34]:
X1= np.array([[0,5,0,0,0,0,0],
    [1,0,5,0,0,0,0],
    [7,0,0,0,0,0,0],
    [0,0,0,13,0,0,0],
    [0,0.5,0,0,0,0,1],
    [1,0,0,0,0,5,1]
    ])
m,n = 6,7

constraints = [
    {(0, 1), (1, 2)},  # First constraint set
    {(2, 0), (3, 3)},  # Second constraint set
]
    
# Corresponding values for constraints
values = [10.0, 20.0]

G, Q, W = construct_constraint_matrices(constraints, values, m, n)

print("Q matrix (row aggregation):")
print(Q)
print("\nG matrix (column aggregation):")
print(G)
print("\nW matrix (constraint values):")
print(W)

# assert np.all(G @ X1 @ Q == W)

Q matrix (row aggregation):
[[0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]]

G matrix (column aggregation):
[[1. 1. 0. 0. 0. 0.]
 [0. 0. 1. 1. 0. 0.]
 [0. 0. 0. 0. 1. 1.]]

W matrix (constraint values):
[[10. nan nan]
 [nan 20. nan]
 [nan nan nan]]


In [39]:
((G @ X1 @ Q) * (1 - np.isnan(W)).astype(float)) == np.where(np.isnan(W), 0, W)

array([[ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True]])

In [77]:
import numpy as np
import scipy.sparse as sp
import warnings

def invd_sparse(x):
    """
    Constructs a diagonal sparse matrix with 1/x, replacing entries where x == 0 with 1.
    
    Parameters:
        x (numpy array or scipy sparse matrix): Input 1D array.
        
    Returns:
        scipy sparse matrix: A diagonal sparse matrix.
    """
    with np.errstate(divide='ignore', invalid='ignore'):
        invd_values = np.where(x != 0, 1.0 / x, 1.0)
    return sp.diags(invd_values.flatten())

def invM_sparse(x):
    """
    Constructs a diagonal sparse matrix with 1/x, replacing entries where x == 0 with 1.
    
    Parameters:
        x (numpy array or scipy sparse matrix): Input 1D array.
        
    Returns:
        scipy sparse matrix: A diagonal sparse matrix.
    """
    with np.errstate(divide='ignore', invalid='ignore'):
        invM_values = np.where( x != 0, 1.0 / x, 1.0)
    return invM_values

def MRGRAS_UpdateMatrix(P, N, T, r, s):
    """
    Updates the matrix using the GRAS balancing approach with sparse matrices.
    
    Parameters:
        P (scipy sparse matrix): Positive part of the input matrix.
        N (scipy sparse matrix): Negative part of the input matrix.
        r (numpy array): Row adjustment factors.
        s (numpy array): Column adjustment factors.
        
    Returns:
        scipy sparse matrix: The updated balanced matrix.
    """
    with np.errstate(divide='ignore', invalid='ignore'):
        invd_T = np.where(T != 0, 1.0 / T, 1.0)
    return sp.diags(r.flatten()) @ (P * T) @ sp.diags(s.flatten()) - invd_sparse(r) @ (N * invd_T) @ invd_sparse(s)

def MRGRAS_Balancing(X, u, v, W, G, Q, EPSILON=1e-10, epsilon_constr = 1e-10 ,max_iter=20):
    # Ensure inputs are consistent
    assert np.allclose(u.sum(), v.sum()), "Row and column sums must be equal."

    # Define matrix size
    m, n = X.shape
    M, N = W.shape
    # Split into positive and negative components (in sparse format)
    P0 = np.maximum(X, 0)
    N0 = np.maximum(-X, 0)
    assert np.allclose(X, (P0 - N0)), "Matrix decomposition into P and N is incorrect."

    # Initialize vectors
    r = np.ones((m, 1))
    t = np.ones((M, N))

    # expand t
    T_mask = (Q @ t @ G).T

    # print(T_mask)

    pr = (T_mask * P0).T @ r        

    with np.errstate(divide='ignore', invalid='ignore'):
        invd_T = np.where(T_mask != 0, 1.0 / T_mask, 1.0)
    nr = ( invd_T * N0).T @ invd_sparse(r) @ np.ones((m, 1))

    # Initial value of s1
    # Ensure pr and nr are compatible for element-wise multiplication
    pr_dense = pr.toarray() if sp.issparse(pr) else pr
    nr_dense = nr.toarray() if sp.issparse(nr) else nr
    s1 = invd_sparse(2 * pr_dense) @ (v + np.sqrt(v ** 2 + 4 * pr_dense * nr_dense))

    # Handle possible NaNs
    s1 = np.nan_to_num(s1, nan=1e-10, posinf=1e-10, neginf=1e-10)

    ss = -invd_sparse(v) @ nr
    s1[pr_dense.flatten() == 0] = ss[pr_dense.flatten() == 0]

    # Iteration loop
    iter = 1
    dif = float('inf')
    t1 = t
    r1 = r

    while (dif > EPSILON) and (iter <= max_iter):
        # Update r
        ps = (P0 * T_mask) @ s1
        ns = (N0 * invd_T) @ invd_sparse(s1) @ np.ones((n, 1))

        ps_dense = ps.toarray() if sp.issparse(ps) else ps
        ns_dense = ns.toarray() if sp.issparse(ns) else ns
        r = invd_sparse(2 * ps_dense) @ (u + np.sqrt(u ** 2 + 4 * ps_dense * ns_dense))
        
        # Handle possible NaNs
        r = np.nan_to_num(r, nan=1e-10, posinf=1e-10, neginf=1e-10)

        rr = -invd_sparse(u) @ ns
        r[ps_dense.flatten() == 0] = rr[ps_dense.flatten() == 0]

        # update t
        pt = G @ np.diag(r.flatten()) @ P0 @ np.diag(s1.flatten()) @ Q 
        nt = G @ invd_sparse(r) @ N0 @ invd_sparse(s1) @ Q 
        
        with np.errstate(divide='ignore', invalid='ignore'):
            pt_invd = np.where(pt != 0, 1.0 / pt, 1.0)
        t = (pt_invd / 2.0)*(W + np.sqrt(W**2 + 4 * pt * nt))

        with np.errstate(divide='ignore', invalid='ignore'):
            W_invd = np.where(W != 0, 1.0 / W, 1.0)
        tr = - nt * W_invd

        t[pt == 0] = tr[ pt == 0]
        # Handle possible NaNs
        t = np.nan_to_num(t, nan=1, posinf=1e-10, neginf=1e-10)


        # expand t
        T_mask = (Q @ t @ G).T

        with np.errstate(divide='ignore', invalid='ignore'):
            invd_T = np.where(T_mask != 0, 1.0 / T_mask, 1.0)
        # Update s2
        pr = (T_mask * P0).T @ r 
        nr = ( invd_T * N0).T @ invd_sparse(r) @ np.ones((m, 1)) # verificare se rimuovere questo np.ones
        
        pr_dense = pr.toarray() if sp.issparse(pr) else pr
        nr_dense = nr.toarray() if sp.issparse(nr) else nr
        s2 = invd_sparse(2 * pr_dense) @ (v + np.sqrt(v ** 2 + 4 * pr_dense * nr_dense))

        # Handle possible NaNs
        s2 = np.nan_to_num(s2, nan=1e-10, posinf=1e-10, neginf=1e-10)
        
        ss = -invd_sparse(v) @ nr
        s2[pr_dense.flatten() == 0] = ss[pr_dense.flatten() == 0]

        # Convergence check
        dif = np.max([np.max(np.abs(s2 - s1)), np.max(np.abs(r - r1)), np.max(np.abs(t - t1))])
        s1 = s2  # Update s1 for next iteration
        r1 = r
        t1 = t
        iter +=1 
    

    # Post iteration checks
    if (iter == max_iter) and (dif > EPSILON ):
        warnings.warn("GRAS procedure did not converge.")
        return None
    else:
        s = s2
        ps = (P0 * T_mask) @ s
        ns = (N0 * invd_T) @ invd_sparse(s) @ np.ones((n, 1))
        
        ps_dense = ps.toarray() if sp.issparse(ps) else ps
        ns_dense = ns.toarray() if sp.issparse(ns) else ns
        r = invd_sparse(2 * ps_dense) @ (u + np.sqrt(u ** 2 + 4 * ps_dense * ns_dense))

        # Handle possible NaNs
        r = np.nan_to_num(r, nan=1e-10, posinf=1e-10, neginf=1e-10)
        
        rr = -invd_sparse(u) @ ns
        r[ps_dense.flatten() == 0] = rr[ps_dense.flatten() == 0]

        X_MRGRAS = MRGRAS_UpdateMatrix(P0, N0, T_mask, r, s)
        row_sums = np.array(X_MRGRAS.sum(axis=1)).flatten()
        column_sums = np.array(X_MRGRAS.sum(axis=0)).flatten()
        if not (np.allclose(row_sums, u.flatten(), atol=epsilon_constr) and np.allclose(column_sums, v.flatten(), atol=epsilon_constr) and np.allclose(G @ X_MRGRAS @ Q, W)):
            warnings.warn("The final balanced matrix does not match the target row/column sums within the specified tolerance.")

        # Final updated matrix
        return X_MRGRAS, iter, dif, r,s,t


In [78]:
np.max([0,1,2])

np.int64(2)

In [107]:

# Test Case 1: Small Basic Matrix / single cell constraint
print("Test Case 1: Small Basic Matrix")
X1 = np.array([[5, -3, 0,1], [2, 1, -2,1], [-1, 3, 4,1]])
u1 = np.array([5, 3, 1]).reshape(-1, 1)
v1 = np.array([3, 4, 1,1]).reshape(-1, 1)

constraints = [
    {(0, 0)},  # First constraint set
    {(1,1)}
    ]
    
# Corresponding values for constraints
values = [6,3]

# Specify input matrix dimensions
m, n = X1.shape  # Dimensions of input matrix X

# Construct matrices
G, Q, W = construct_constraint_matrices(constraints, values, m, n)
result1, iterations1, diff1,r,s,t = MRGRAS_Balancing(X1, u1,v1, W, G, Q, EPSILON=1e-14, epsilon_constr=1e-14, max_iter=200)

assert result1 is not None, "Test Case 1 failed: Procedure did not converge."
assert np.allclose(result1.sum(axis = 1).flatten(), u1.flatten(), atol=1e-10), "Test Case 1 failed: Row sum target not reached"
assert np.allclose(result1.sum(axis = 0).flatten(), v1.flatten(), atol=1e-10), "Test Case 1 failed: Columns sum target not reached"
assert np.allclose((G @ result1 @ Q) * (~np.isnan(W)).astype(int) , np.where(np.isnan(W), 0, W), atol = 1e-10), "Test Case 1 failed: Constraint sum target not reached"

# Check if the sign of each element is preserved
assert np.all((X1 >= 0) == (result1 >= 0)), "Test Case 1 failed: Signs of positive cells are not preserved."
assert np.all((X1 <= 0) == (result1 <= 0)), "Test Case 1 failed: Signs of negative cells are not preserved."
print(f"Test Case 1 passed: Converged in {iterations1} iterations with final difference {diff1}")

Test Case 1: Small Basic Matrix
Test Case 1 passed: Converged in 201 iterations with final difference 1.9984014443252818e-14


C:\Users\Raul De Maio\AppData\Local\Temp\ipykernel_9800\3001350975.py:177: UserWarning: The final balanced matrix does not match the target row/column sums within the specified tolerance.
  warnings.warn("The final balanced matrix does not match the target row/column sums within the specified tolerance.")


In [108]:
# Test Case 2: Diagonal Matrix
print("\nTest Case 2: Diagonal Matrix")
X2 = np.array([[5, 0, 0], [0, -4, 0], [0, 0, 2]])
u2 = np.array([5, -4, 3]).reshape(-1, 1)
v2 = np.array([5, -4, 3]).reshape(-1, 1)

constraints = [
    {(0, 0)},  # First constraint set
    {(1, 1)}  # Second constraint set
    ]
    
# Corresponding values for constraints
values = [5.0,-4.0]

# Specify input matrix dimensions
m, n = X2.shape  # Dimensions of input matrix X

G, Q, W = construct_constraint_matrices(constraints, values, m, n)
result2, iterations2, diff2,r,s,t = MRGRAS_Balancing(X2, u2, v2, W, G, Q, max_iter= 200)
assert result2 is not None, "Test Case 2 failed: Procedure did not converge."
# Check if the sign of each element is preserved
assert np.all((X2 >= 0) == (result2 >= 0)), "Test Case 2 failed: Signs of positive cells are not preserved."
assert np.all((X2 <= 0) == (result2 <= 0)), "Test Case 2 failed: Signs of negative cells are not preserved."
print(f"Test Case 2 passed: Converged in {iterations2} iterations with final difference {diff2}")


Test Case 2: Diagonal Matrix
Test Case 2 passed: Converged in 2 iterations with final difference 0.0


C:\Users\Raul De Maio\AppData\Local\Temp\ipykernel_9800\3001350975.py:177: UserWarning: The final balanced matrix does not match the target row/column sums within the specified tolerance.
  warnings.warn("The final balanced matrix does not match the target row/column sums within the specified tolerance.")


In [86]:
# Test Case 3: All Positive Matrix
print("\nTest Case 3: All Positive Matrix")
X3 = np.array([[3, 2, 0], [4, 5, 0], [0, 0, 9]])
u3 = np.array([9, 16, 11]).reshape(-1, 1)
v3 = np.array([5, 20, 11]).reshape(-1, 1)

constraints = [
    {(0, 0),(0,1),(1,0),(1,1)},  # First constraint set
    {(2,2)}
    ]
    
# Corresponding values for constraints
values = [25,11]
G, Q, W = construct_constraint_matrices(constraints, values, m, n)



result3, iterations3, diff3,r,s,t = MRGRAS_Balancing(X3, u3, v3,W,G,Q)
assert result3 is not None, "Test Case 3 failed: Procedure did not converge."
# Check if the sign of each element is preserved
assert np.all((X3 >= 0) == (result3 >= 0)), "Test Case 3 failed: Signs of positive cells are not preserved."
print(f"Test Case 3 passed: Converged in {iterations3} iterations with final difference {diff3}")



Test Case 3: All Positive Matrix
Test Case 3 passed: Converged in 8 iterations with final difference 2.176925306685007e-12


C:\Users\Raul De Maio\AppData\Local\Temp\ipykernel_9800\3001350975.py:177: UserWarning: The final balanced matrix does not match the target row/column sums within the specified tolerance.
  warnings.warn("The final balanced matrix does not match the target row/column sums within the specified tolerance.")


In [83]:
# Test Case 3: All Positive Matrix
print("\nTest Case 3: All Positive Matrix")
X3 = np.array([[3, 2, 0], [4, 5,10], [1, 9,7]])
u3 = np.array([9, 16, 12,]).reshape(-1, 1)
v3 = np.array([5, 20, 12]).reshape(-1, 1)

constraints = [
    {(0, 0)},  # First constraint set
    {(2,2)}
    ]
    
# Corresponding values for constraints
values = [4,0]
m,n = X3.shape
G, Q, W = construct_constraint_matrices(constraints, values, m, n)



result3, iterations3, diff3,r,s,t = MRGRAS_Balancing(X3, u3, v3,W,G,Q, max_iter=2000)
assert result3 is not None, "Test Case 3 failed: Procedure did not converge."
# Check if the sign of each element is preserved
assert np.all((X3 >= 0) == (result3 >= 0)), "Test Case 3 failed: Signs of positive cells are not preserved."
print(f"Test Case 3 passed: Converged in {iterations3} iterations with final difference {diff3}")



Test Case 3: All Positive Matrix
Test Case 3 passed: Converged in 115 iterations with final difference 9.82729453369302e-11


C:\Users\Raul De Maio\AppData\Local\Temp\ipykernel_9800\3001350975.py:177: UserWarning: The final balanced matrix does not match the target row/column sums within the specified tolerance.
  warnings.warn("The final balanced matrix does not match the target row/column sums within the specified tolerance.")


In [117]:
N = 1000

X_test = np.random.rand(N,N)

u = np.ones(N).reshape(-1,1)*4
v = np.ones(N).reshape(-1,1)*4

constraints = [
    {(0, 0),(0,1),(0,2),(1,0),(1,1),(1,2),(2,0),(2,1),(2,2)}
    ]
values = [9]

G, Q, W = construct_constraint_matrices(constraints, values, N, N)

result, iterations, diff,r,s,t = MRGRAS_Balancing(X_test, u, v, W, G, Q, max_iter=2000)

C:\Users\Raul De Maio\AppData\Local\Temp\ipykernel_9800\3001350975.py:177: UserWarning: The final balanced matrix does not match the target row/column sums within the specified tolerance.
  warnings.warn("The final balanced matrix does not match the target row/column sums within the specified tolerance.")


In [118]:
result

array([[1.50672790e-01, 1.07406608e+00, 1.13672787e+00, ...,
        2.06479654e-03, 8.79494912e-04, 1.08764951e-03],
       [9.45829676e-01, 1.54658317e+00, 6.28372979e-01, ...,
        1.35432673e-03, 8.00244689e-04, 1.61421861e-03],
       [1.73858424e+00, 7.54372250e-01, 1.02479095e+00, ...,
        2.41316481e-04, 3.91188623e-04, 4.42475038e-04],
       ...,
       [4.91709602e-04, 6.53186338e-04, 1.44991233e-03, ...,
        5.84778843e-03, 1.41619333e-03, 3.18643821e-03],
       [5.80953742e-04, 1.13885235e-03, 1.69996458e-03, ...,
        6.81883707e-03, 7.09379340e-03, 7.77236855e-03],
       [1.00225846e-03, 1.13945033e-04, 1.84431065e-03, ...,
        7.65857096e-03, 6.37784059e-03, 2.55114925e-04]])

In [119]:
import pandas as pd

pd.DataFrame(result).to_clipboard()